In [205]:
import os
from tika import parser
import csv

In [209]:
filelist = os.listdir('pdfs')

In [97]:
def extractBigEntity(textList):
    checkpoint = 0
    statList = []
    for i in range(6):
        stoppoint = textList[checkpoint+1:].index('compared to baseline')
        returnSet = textList[checkpoint:checkpoint+stoppoint+1]
        statList.append(returnSet[-1])
        checkpoint += stoppoint + 1
    return statList, checkpoint

In [217]:
statList, checkpoint = extractBigEntity(text)
print(statList, checkpoint)
statList.insert(0,'aaaaa')
print(statList)

['-39%', '+3%', '+11%', '-35%', '-29%', '+10%'] 72
['aaaaa', '-39%', '+3%', '+11%', '-35%', '-29%', '+10%']


In [172]:
def extractSmallEntity(textList):
    categories = ['Retail & recreation', 'Grocery & pharmacy', 'Parks', 'Transit stations', 'Workplace', 'Residential']
    countyDict = {}
    statList = []
    county = ""
    for i in range(len(textList)):
        if len(statList) == 6 and county != "":
            countyDict[county] = statList
            statList = []
            county = ""
        for element in categories:
            if element in textList[i]:
                if "Retail & recreation" in textList[i]:
                    county = textList[i-1]
                    #print(county)
                if 'compared to baseline' in textList[i+1]:
                    splitLine = textList[i+1].split(' ')
                    #print(splitLine[0])
                    statList.append(splitLine[0])
                    #print(textList[i])
                elif 'Not enough data for this date' in textList[i+1]:
                    statList.append("N/A")
                    #print("Not enough data")
                    #print(textList[i])
    return countyDict

In [173]:
result = extractSmallEntity(text[72:])
for element in result:
    print(element, result[element])
    print()

Autauga County ['-37%', '+16%', '-14%', 'N/A', '-30%', '+20%']

Baldwin County ['-44%', '-10%', '-42%', '-39%', '-28%', '+11%']

Barbour County ['-18%', '-20%', 'N/A', 'N/A', '-31%', 'N/A']

Bibb County ['-42%', '-15%', 'N/A', 'N/A', '-19%', 'N/A']

Blount County ['-20%', '+18%', 'N/A', 'N/A', '-24%', '+20%']

Bullock County ['-22%', '-27%', 'N/A', 'N/A', '-33%', 'N/A']

Butler County ['-47%', '-9%', 'N/A', '-57%', '-33%', 'N/A']

Calhoun County ['-40%', '+1%', '-22%', 'N/A', '-29%', '+9%']

Chambers County ['-35%', '-3%', 'N/A', '-40%', '-32%', '+15%']

Cherokee County ['-41%', '-5%', 'N/A', 'N/A', '-18%', 'N/A']

Chilton County ['-28%', '+11%', 'N/A', '-30%', '-25%', '+17%']

Choctaw County ['-13%', '-19%', 'N/A', 'N/A', '-43%', 'N/A']

Clarke County ['-22%', '-29%', 'N/A', 'N/A', '-9%', 'N/A']

Clay County ['-43%', '-20%', 'N/A', 'N/A', '-41%', 'N/A']

Cleburne County ['-35%', '-13%', 'N/A', '-42%', '-14%', 'N/A']

Coffee County ['-25%', '-2%', 'N/A', 'N/A', '-27%', '+18%']

Colbert

In [200]:
def extractLocationName(filename):
    filenamesplit = filename.split('_')
    if(filenamesplit[2] == 'Mobility'):
        return "US"
    else:
        return filenamesplit[2]

In [219]:
def convertPDFtoCSV(filename):
    location = extractLocationName(filename)
    
    csvfilename = location+'.csv'
    csvfile = open('CSV/'+csvfilename, 'w')
    writer = csv.writer(csvfile)
    writer.writerow(['X','Retail & recreation', 'Grocery & pharmacy', 'Parks', 'Transit stations', 'Workplace', 'Residential'])
    
    pdf = parser.from_file("pdfs/"+filename)
    text = pdf['content'].strip().split('\n')
    for element in text:
        if element == '':
            text.remove(element)
    overview, checkpoint = extractBigEntity(text)
    overview.insert(0,location)
    writer.writerow(overview)
    resultDict = extractSmallEntity(text[checkpoint:])
    for key in resultDict:
        resultList = resultDict[key]
        resultList.insert(0,key)
        writer.writerow(resultList)
    csvfile.close()

In [221]:
for filename in filelist:
    convertPDFtoCSV(filename)